# Record Embedding

In [1]:
import pandas as pd
import numpy as np
import os
import calendar
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

## Read Data
We will be using a dataset containing book data from Amazon. I scraped this dataset off Amazon. It contains about 3400 tuples of various datatypes.

## Exploration using the Gensim library
Details here: https://radimrehurek.com/gensim/

In [2]:
import warnings
warnings.filterwarnings('ignore')

from gensim.models import Word2Vec
import gensim

Convert each tuple into a row

In [8]:
len(combined)

7604

In [3]:
#hospital dataset

#1. load hospital dataset and turthvalue.csv; mark error for unmatched values and cleanup using FastText; fill empty cells with word2vec
#finding errors and outliers -> discover FDs and try marking errors? or word2vec most_similar of val and if row does not contain the cell values??!

df_hospital = pd.read_csv('dirty_data_transformed.csv', dtype=object)
df_truth = pd.read_csv('truthvalue.csv', dtype=object)



provider = df_truth['ProviderNumber'].tolist()
hospital = df_truth['HospitalName'].tolist()
address = df_truth['Address1'].tolist()
city = df_truth['City'].tolist()
state = df_truth['State'].tolist()
zipcode = df_truth['ZipCode'].tolist()
county = df_truth['CountyName'].tolist()
phone = df_truth['PhoneNumber'].tolist()
hospType = df_truth['HospitalType'].tolist()
owner = df_truth['HospitalOwner'].tolist()
service = df_truth['EmergencyService'].tolist()
condition = df_truth['Condition'].tolist()
code = df_truth['MeasureCode'].tolist()
name = df_truth['MeasureName'].tolist()
score = df_truth['Score'].tolist()
sample = df_truth['Sample'].tolist()
stateavg = df_truth['Stateavg'].tolist()

combined_hosp = list(zip(provider, hospital, address, city, state, zipcode, county, phone, hospType, owner, service, condition, code, name, score, sample, stateavg))



In [25]:
df_truth.head()

,ProviderNumber,HospitalName,Address1,City,State,ZipCode,CountyName,PhoneNumber,HospitalType,HospitalOwner,EmergencyService,Condition,MeasureCode,MeasureName,Score,Sample,Stateavg
0,10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-CARD-2,surgery patients who were taking heart drugs c...,Empty,Empty,AL_SCIP-CARD-2
1,10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-INF-1,surgery patients who were given an antibiotic ...,Empty,Empty,AL_SCIP-INF-1
2,10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-INF-2,surgery patients who were given the right kin...,Empty,Empty,AL_SCIP-INF-2
3,10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-INF-3,surgery patients whose preventive antibiotics ...,Empty,Empty,AL_SCIP-INF-3
4,10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-INF-4,all heart surgery patients whose blood sugar (...,Empty,Empty,AL_SCIP-INF-4


In [5]:
#hospital dirty dataset; maintain dirty dataset separately
provider = df_hospital['ProviderNumber'].tolist()
hospital = df_hospital['HospitalName'].tolist()
address = df_hospital['Address1'].tolist()
city = df_hospital['City'].tolist()
state = df_hospital['State'].tolist()
zipcode = df_hospital['ZipCode'].tolist()
county = df_hospital['CountyName'].tolist()
phone = df_hospital['PhoneNumber'].tolist()
hospType = df_hospital['HospitalType'].tolist()
owner = df_hospital['HospitalOwner'].tolist()
service = df_hospital['EmergencyService'].tolist()
condition = df_hospital['Condition'].tolist()
code = df_hospital['MeasureCode'].tolist()
name = df_hospital['MeasureName'].tolist()
score = df_hospital['Score'].tolist()
sample = df_hospital['Sample'].tolist()
stateavg = df_hospital['Stateavg'].tolist()

combined_dirty = list(zip(provider, hospital, address, city, state, zipcode, county, phone, hospType, owner, service, condition, code, name, score, sample, stateavg))
len(combined_dirty)




1000

In [6]:
#Calculating error or dirty cells
dirty_list = []
truth_list = []
for i in range(len(combined_dirty)):
    for j in range(len(combined_dirty[0])):
        if combined_dirty[i][j] != combined_hosp[i][j]:
            dirty_list.append(combined_dirty[i][j])
            truth_list.append(combined_hosp[i][j])

In [7]:
# train model
bigram_transformer = gensim.models.Phrases(combined_hosp, delimiter=b' ')
model = Word2Vec(bigram_transformer[combined_hosp], sg=1, min_count=1, workers=8, iter=1000)

In [8]:
print(model)

Word2Vec(vocab=663, size=100, alpha=0.025)


In [20]:
type(dirty_list[1])


str

In [10]:
#module to predict the top value by the model.Some values are not broken into tokens.
#Values with keyerrors are stored in excluded_list

excluded_list = []
result = []
for i in range(len(dirty_list)):
    if isinstance(dirty_list[i], str):
        try:
            result.append((model.most_similar(dirty_list[i])[0][0],dirty_list[i],truth_list[i]))
        except KeyError:
            excluded_list.append(dirty_list[i])
            #print(dirty_list[i])
            pass

In [11]:
print (len(excluded_list))
print (len(result))
print (len(excluded_list) + len(result))
print (len(dirty_list))
excluded_list[0]

560
34
594
594


'surgery patients who were taking heart drugs caxxed beta bxockers before coming to the hospitax who were kept on the beta bxockers during the period just before and after their surgery'

In [12]:
#identify the true positives and false positives. Store them in the seperate list for further analysis.
true_pos = 0
false_pos = 0
correctly_predicted = []
falsely_predicted = []
for i in range(len(result)):
    if result[i][0] == result[i][2]:
        true_pos = true_pos + 1
        correctly_predicted.append(result[i])
    else:
        false_pos = false_pos + 1
        falsely_predicted.append(result[i])

In [23]:
type(dirty_list[0])

str

100 dimensional vectors.

In [24]:
model.most_similar("surgery patients who were taking heart drugs caxxed beta bxockers before coming to the hospitax who were kept on the beta bxockers during the period just before and after their surgery")

KeyError: "word 'surgery patients who were taking heart drugs caxxed beta bxockers before coming to the hospitax who were kept on the beta bxockers during the period just before and after their surgery' not in vocabulary"

In [23]:
model.save("amazonModelWord2Vec.w2v")